<a href="https://colab.research.google.com/github/nullpitch-dev/DS_L1_Notebooks/blob/master/DS_L1_EX_03.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd

# loading data from a csv fle
url = 'https://raw.githubusercontent.com/nullpitch-dev/hj_public/master/Antibiotic_70K_patinets.csv'
data = pd.read_csv(url)

In [6]:
# [1]

data.describe()

,ID,total_d,high_d,low_d,none_high_d,high_p,cdiff,age_5
count,70000.000000,70000.000000,70000.000000,70000.000000,70000.000000,70000.000000,70000.000000,70000.00000
mean,35000.500000,8.049771,3.774386,6.131871,4.275386,42.254031,0.021471,2.74870
std,20207.403759,13.885638,8.795789,11.502711,7.950063,39.836986,0.144951,1.56102
min,1.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.00000
25%,17500.750000,2.000000,0.000000,2.000000,1.000000,0.000000,0.000000,1.00000
50%,35000.500000,4.000000,2.000000,3.000000,2.000000,37.500000,0.000000,3.00000
75%,52500.250000,8.000000,4.000000,6.000000,4.000000,80.000000,0.000000,4.00000
max,70000.000000,526.000000,368.000000,397.000000,294.000000,100.000000,1.000000,7.00000


In [69]:
# [2]

data2 = data.groupby('age_5').agg({'high_p': 'mean'})
data2.sort_values(by='high_p', ascending=False, inplace=True)

print(f'Answer [2] : Top age_5 = {data2.index[0]}')

Answer [2] : Top age_5 = 7


In [70]:
# [3]

import math

check_na = data[data.isna()]

ll = math.floor(data['total_d'].mean() * 100) / 100 -\
     3 * math.floor(data['total_d'].std() * 100) / 100
ul = math.floor(data['total_d'].mean() * 100) / 100 +\
     3 * math.floor(data['total_d'].std() * 100) / 100

data3 = data.assign(outlier=data.apply(lambda x:
                                       1 if (x['total_d'] < ll) | (x['total_d'] > ul)
                                       else 0, axis=1))

print(f'Answer [3] : No of null = {sum(check_na.sum()):.0f}, ' +
      f"No of outliers = {data3['outlier'].sum()}")

Answer [3] : No of null = 0, No of outliers = 1258


In [72]:
# [4]

import math

corr = data[['age_5', 'high_p']].corr(method='pearson')['age_5'].iloc[1]

print(f'Answer [4] : Corr = {math.floor(corr * 100) / 100}')

Answer [4] : Corr = 0.12


In [95]:
# [5] ttest_ind

from scipy import stats

yes = data[data['cdiff'] == 1]
no = data[data['cdiff'] == 0]

diff = math.floor((yes['high_p'].mean() - no['high_p'].mean()) * 100) / 100

t_val, p_val = stats.ttest_ind(yes['high_p'], no['high_p'])

print(f'Answer [5] : Difference = {diff}, P-value = {p_val}')
print(f'             P-value is less than 0.05 so the means are not same')

Answer [5] : Difference = 4.79, P-value = 3.975323449530875e-06
             P-value is less than 0.05 so the means are not same


In [131]:
# [6] LogisticRegression

from sklearn.linear_model import LogisticRegression

train_X = data[['total_d', 'high_d', 'low_d', 'high_p', 'age_5']]
train_y = data['cdiff']

lr = LogisticRegression(solver='newton-cg')
model = lr.fit(train_X, train_y)
coef = model.coef_

ind = []
for i in range(len(coef[0])):
    if coef[0, i] < 0:
        ind.append(i)

ans = f'Answer [6] : '
for i in range(len(ind)):
    ans += f'{train_X.columns[ind][i]} '

print(ans)

Answer [6] : high_d low_d 
